# Graph API Readings from Raspberry Pi
This notebook fetches sensor readings from the API and plots them.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# Set your API endpoint here
API_URL = 'http://raspberrypi.middlebury.edu:5000/readings'  # Change if needed

# Fetch the readings
response = requests.get(API_URL)
response.raise_for_status()
data = response.json()

# Display the raw JSON (optional)
data

In [ ]:
# Convert to DataFrame (adjust based on your API's JSON structure)
# For example, if data is a list of dicts with 'timestamp', 'value'
df = pd.DataFrame(data)

# If timestamps are present, parse them
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values('timestamp')

df.head()

In [ ]:
# Plot the readings
plt.figure(figsize=(12, 6))
if 'timestamp' in df.columns:
    plt.plot(df['timestamp'], df['value'], marker='o')
    plt.xlabel('Timestamp')
else:
    plt.plot(df['value'], marker='o')
    plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Sensor Readings from API')
plt.grid(True)
plt.show()